# Notebook 02: Glue & Athena Data Catalog


In [1]:
import os
import sys
import time

import boto3
import pandas as pd
import json
import pyarrow as pa
import pyarrow.parquet as pq
from botocore.exceptions import ClientError

In [2]:
# Add project root to path
sys.path.append("..")

#load config
from config.config import (
    BUCKET_NAME,
    AWS_REGION,
    S3_PREFIX,
    RAW_DATA_FILENAME,
    GLUE_DATABASE_NAME,
    GLUE_CRAWLER_NAME,
    get_s3_uri,
)

print(f"Bucket: {BUCKET_NAME}")
print(f"Region: {AWS_REGION}")
print(f"Glue Database: {GLUE_DATABASE_NAME}")
print(f"Glue Crawler: {GLUE_CRAWLER_NAME}")

Bucket: nfci-forecasting-306617143793
Region: us-east-1
Glue Database: nfci_database
Glue Crawler: nfci-raw-data-crawler


### create glue and athena AWS clients

In [3]:
# Create clients
glue_client = boto3.client("glue", region_name=AWS_REGION)
athena_client = boto3.client("athena", region_name=AWS_REGION)

print("Glue client created")
print("Athena client created")

Glue client created
Athena client created


### Convert CSV to Parquet

In [4]:
def convert_csv_to_parquet(
    csv_s3_uri: str,
    parquet_s3_uri: str,
    date_columns: list = None
) -> bool:
    """
    Read CSV from S3, convert to Parquet, and upload back to S3.
    
    Parameters:
        csv_s3_uri: S3 path to source CSV file
        parquet_s3_uri: S3 path for output Parquet file
        date_columns: List of columns to parse as dates
    
    Returns:
        True if successful
    """
    print(f"Reading CSV from: {csv_s3_uri}")
    
    # Read CSV with appropriate parsing
    df = pd.read_csv(
        csv_s3_uri,
        parse_dates=date_columns or []
    )
    
    print(f"  Shape: {df.shape}")
    
    # Convert to Parquet and upload to S3
    print(f"\nWriting Parquet to: {parquet_s3_uri}")
    
    df.to_parquet(
        parquet_s3_uri,
        engine="pyarrow",
        compression="snappy",  # Good balance of speed and compression
        index=False
    )
    
    print(f"Parquet file created successfully")
    return True

In [5]:
# Define source and destination paths
csv_source = get_s3_uri("raw", RAW_DATA_FILENAME)
parquet_filename = RAW_DATA_FILENAME.replace(".csv", ".parquet")
parquet_dest = f"s3://{BUCKET_NAME}/data/raw_parquet/{parquet_filename}"

print(f"Source CSV:      {csv_source}")
print(f"Destination:     {parquet_dest}")

# Convert CSV to Parquet
convert_csv_to_parquet(
    csv_s3_uri=csv_source,
    parquet_s3_uri=parquet_dest,
    date_columns=["date"]
)

Source CSV:      s3://nfci-forecasting-306617143793/data/raw/state_month_full.csv
Destination:     s3://nfci-forecasting-306617143793/data/raw_parquet/state_month_full.parquet
Reading CSV from: s3://nfci-forecasting-306617143793/data/raw/state_month_full.csv
  Shape: (12000, 42)

Writing Parquet to: s3://nfci-forecasting-306617143793/data/raw_parquet/state_month_full.parquet
Parquet file created successfully


True

In [6]:
s3_client = boto3.client("s3", region_name=AWS_REGION)

# Get Parquet size
parquet_key = f"{S3_PREFIX['raw']}/{parquet_filename}"
parquet_obj = s3_client.head_object(Bucket=BUCKET_NAME, Key=parquet_key)
parquet_size = parquet_obj["ContentLength"] / 1024 / 1024  # MB

print(f"Parquet size:    {parquet_size:.2f} MB")

Parquet size:    0.11 MB


In [7]:
## check parquet file
df_verify = pd.read_parquet(parquet_dest)
for col, dtype in df_verify.dtypes.items():
    print(f"  {col:<25} {str(dtype):<15}")

  state_fips                int64          
  date                      datetime64[ns] 
  UNRATE                    float64        
  PAYEMS                    int64          
  CIVPART                   float64        
  EMRATIO                   float64        
  U6RATE                    float64        
  AWHMAN                    float64        
  AHETPI                    float64        
  CPIAUCSL                  float64        
  CPILFESL                  float64        
  PCEPI                     float64        
  PCEPILFE                  float64        
  PPIFDG                    float64        
  INDPRO                    float64        
  RRSFS                     float64        
  DGORDER                   float64        
  UMCSENT                   float64        
  HOUST                     float64        
  PERMIT                    float64        
  CSUSHPINSA                float64        
  MSPUS                     float64        
  FEDFUNDS                  floa

### create glue database

In [8]:
try:
    glue_client.create_database(
        DatabaseInput={
            "Name": GLUE_DATABASE_NAME,
            "Description": "Database for NFCI Forecasting Project",
        }
    )
    print(f"Database '{GLUE_DATABASE_NAME}' created successfully")
except ClientError as e:
    if e.response["Error"]["Code"] == "AlreadyExistsException":
        print(f"Database '{GLUE_DATABASE_NAME}' already exists")
    else:
        print(f"Error creating database: {e}")

Database 'nfci_database' created successfully


### Create Glue Crawler
- create glue IAM role and policy
- 

In [9]:
import json

# IAM client
iam_client = boto3.client("iam")

# Role name for Glue
GLUE_ROLE_NAME = "nfci-glue-crawler-role"

- create Glue IAM role

In [10]:
def create_glue_role(role_name: str, bucket_name: str) -> str:
    """
    Create an IAM role for Glue Crawler with necessary permissions.
    
    Returns the role ARN.
    """
    
    # Trust policy - allows Glue service to assume this role
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "glue.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    # S3 policy - allows access to our specific bucket
    s3_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:PutObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*"
                ]
            }
        ]
    }
    
    try:
        # Create the role
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="IAM role for Glue Crawler - NFCI project"
        )
        role_arn = response["Role"]["Arn"]
        print(f"✓ Created role: {role_name}")
        
    except ClientError as e:
        if e.response["Error"]["Code"] == "EntityAlreadyExists":
            # Role exists, get its ARN
            response = iam_client.get_role(RoleName=role_name)
            role_arn = response["Role"]["Arn"]
            print(f"✓ Role already exists: {role_name}")
        else:
            raise e
    
    # Attach the AWS managed Glue service policy
    try:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn="arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole"
        )
        print(f"Attached AWSGlueServiceRole policy")
    except ClientError as e:
        if "already" not in str(e).lower():
            print(f"  Note: {e}")
    
    # Create and attach custom S3 policy for our bucket
    s3_policy_name = f"{role_name}-s3-policy"
    
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=s3_policy_name,
            PolicyDocument=json.dumps(s3_policy)
        )
        print(f"Attached S3 access policy for bucket: {bucket_name}")
    except ClientError as e:
        print(f"  Note: {e}")
    
    return role_arn

In [11]:
# Create the Glue role
GLUE_ROLE_ARN = create_glue_role(GLUE_ROLE_NAME, BUCKET_NAME)
print(f"\nGlue Role ARN: {GLUE_ROLE_ARN}")

✓ Role already exists: nfci-glue-crawler-role
Attached AWSGlueServiceRole policy
Attached S3 access policy for bucket: nfci-forecasting-306617143793

Glue Role ARN: arn:aws:iam::306617143793:role/nfci-glue-crawler-role


#### create crawler

In [12]:
def create_glue_crawler(
    crawler_name: str,
    role_arn: str,
    database_name: str,
    s3_target_path: str,
    table_prefix: str = ""
) -> bool:
    """
    Create a Glue Crawler to catalog data in S3.
    
    Parameters:
        crawler_name: Name for the crawler
        role_arn: IAM role ARN with Glue and S3 permissions
        database_name: Target Glue database for discovered tables
        s3_target_path: S3 path to crawl (e.g., s3://bucket/prefix/)
        table_prefix: Optional prefix for table names
    """
    try:
        glue_client.create_crawler(
            Name=crawler_name,
            Role=role_arn,
            DatabaseName=database_name,
            Description=f"Crawler for {s3_target_path}",
            Targets={
                "S3Targets": [
                    {
                        "Path": s3_target_path,
                        "Exclusions": []  # No exclusions
                    }
                ]
            },
            TablePrefix=table_prefix,
            SchemaChangePolicy={
                "UpdateBehavior": "UPDATE_IN_DATABASE",  # Update existing tables
                "DeleteBehavior": "LOG"  # Log deleted objects, don't remove from catalog
            },
            RecrawlPolicy={
                "RecrawlBehavior": "CRAWL_EVERYTHING"  # Re-crawl all data each time
            },
            Configuration='{"Version":1.0,"CrawlerOutput":{"Partitions":{"AddOrUpdateBehavior":"InheritFromTable"}}}'
        )
        print(f"Crawler '{crawler_name}' created successfully")
        return True
        
    except ClientError as e:
        if e.response["Error"]["Code"] == "AlreadyExistsException":
            print(f"Crawler '{crawler_name}' already exists")
            return True
        else:
            print(f"Error creating crawler: {e}")
            return False

In [13]:
# Build S3 path for raw data Parquet file
raw_data_s3_path = f"s3://{BUCKET_NAME}/data/raw_parquet/"
print(f"Crawler will scan: {raw_data_s3_path}")

# Create the crawler
create_glue_crawler(
    crawler_name=GLUE_CRAWLER_NAME,
    role_arn=GLUE_ROLE_ARN,
    database_name=GLUE_DATABASE_NAME,
    s3_target_path=raw_data_s3_path,
    table_prefix=""  # No prefix - table will be named based on file
)

Crawler will scan: s3://nfci-forecasting-306617143793/data/raw_parquet/
Crawler 'nfci-raw-data-crawler' created successfully


True

#### Run crawler

In [14]:
def run_crawler_and_wait(crawler_name: str, timeout_minutes: int = 10) -> bool:
    """
    Start a Glue Crawler and wait for it to complete.
    Parameters:
        crawler_name: Name of the crawler to run
        timeout_minutes: Maximum time to wait
    
    Returns:
        True if crawler completed successfully
    """
    # Start the crawler
    try:
        glue_client.start_crawler(Name=crawler_name)
        print(f"Crawler '{crawler_name}' started")
    except ClientError as e:
        if "CrawlerRunningException" in str(e):
            print(f"Crawler '{crawler_name}' is already running")
        else:
            print(f"Error starting crawler: {e}")
            return False
    
    # Poll for completion
    print("\nWaiting for crawler to complete...")
    timeout_seconds = timeout_minutes * 60
    start_time = time.time()
    
    while True:
        # Check elapsed time
        elapsed = time.time() - start_time
        if elapsed > timeout_seconds:
            print(f"\nTimeout after {timeout_minutes} minutes")
            return False
        
        # Get crawler status
        response = glue_client.get_crawler(Name=crawler_name)
        state = response["Crawler"]["State"]
        
        if state == "READY":
            # Check last crawl status
            last_crawl = response["Crawler"].get("LastCrawl", {})
            status = last_crawl.get("Status", "UNKNOWN")
            
            if status == "SUCCEEDED":
                print(f"\nCrawler completed successfully!")
                print(f"  Log: {last_crawl.get('LogGroup')}/{last_crawl.get('LogStream')}")
                return True
            else:
                error_msg = last_crawl.get("ErrorMessage", "Unknown error")
                print(f"\nCrawler failed: {status}")
                print(f"  Error: {error_msg}")
                return False
        
        # Still running
        print(f" Status: {state} (elapsed: {int(elapsed)}s)", end="\r")
        time.sleep(10)  # Check every 10 seconds

In [15]:
# Run the crawler
run_crawler_and_wait(GLUE_CRAWLER_NAME)

Crawler 'nfci-raw-data-crawler' started

Waiting for crawler to complete...
 Status: STOPPING (elapsed: 111s)
Crawler completed successfully!
  Log: /aws-glue/crawlers/nfci-raw-data-crawler


True

#### verify table creation

In [16]:
def list_tables_in_database(database_name: str) -> list:
    """List all tables in a Glue database."""
    response = glue_client.get_tables(DatabaseName=database_name)
    tables = response.get("TableList", [])
    return [t["Name"] for t in tables]

In [17]:
# List tables
tables = list_tables_in_database(GLUE_DATABASE_NAME)
print(f"Tables in '{GLUE_DATABASE_NAME}':\n")
for table in tables:
    print(f"  - {table}")

Tables in 'nfci_database':

  - raw_parquet


In [18]:
def get_table_schema(database_name: str, table_name: str) -> None:
    """Display schema for a Glue table."""
    response = glue_client.get_table(
        DatabaseName=database_name,
        Name=table_name
    )
    
    table = response["Table"]
    columns = table["StorageDescriptor"]["Columns"]
    location = table["StorageDescriptor"]["Location"]
    
    print(f"Table: {table_name}")
    print(f"Location: {location}")
    print(f"\nColumns ({len(columns)} total):\n")
    print(f"{'Column Name':<25} {'Data Type':<15}")
    print("-" * 40)
    
    for col in columns:
        print(f"{col['Name']:<25} {col['Type']:<15}")

In [20]:
# Get schema for the raw data table
if tables:
    raw_table_name = "raw_parquet"  # Assuming first table is our raw data
    get_table_schema(GLUE_DATABASE_NAME, raw_table_name)

Table: raw_parquet
Location: s3://nfci-forecasting-306617143793/data/raw_parquet/

Columns (42 total):

Column Name               Data Type      
----------------------------------------
state_fips                bigint         
date                      bigint         
unrate                    double         
payems                    bigint         
civpart                   double         
emratio                   double         
u6rate                    double         
awhman                    double         
ahetpi                    double         
cpiaucsl                  double         
cpilfesl                  double         
pcepi                     double         
pcepilfe                  double         
ppifdg                    double         
indpro                    double         
rrsfs                     double         
dgorder                   double         
umcsent                   double         
houst                     double         
permit         

### Query Data with Athena

In [21]:
# Athena output location
ATHENA_OUTPUT_LOCATION = f"s3://{BUCKET_NAME}/{S3_PREFIX['athena_results']}/"
print(f"Athena results will be stored at: {ATHENA_OUTPUT_LOCATION}")

Athena results will be stored at: s3://nfci-forecasting-306617143793/athena-results/


- run query function

In [22]:
def run_athena_query(query: str, database: str, output_location: str) -> str:
    """
    Execute an Athena query and return the query execution ID.
    
    Parameters:
        query: SQL query string
        database: Glue database name
        output_location: S3 path for query results
    
    Returns:
        Query execution ID
    """
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": database},
        ResultConfiguration={"OutputLocation": output_location}
    )
    return response["QueryExecutionId"]

- wait for query function

In [23]:
def wait_for_query(query_execution_id: str, timeout_seconds: int = 300) -> str:
    """
    Wait for an Athena query to complete.
    
    Returns:
        Final state: 'SUCCEEDED', 'FAILED', or 'CANCELLED'
    """
    start_time = time.time()
    
    while True:
        elapsed = time.time() - start_time
        if elapsed > timeout_seconds:
            return "TIMEOUT"
        
        response = athena_client.get_query_execution(
            QueryExecutionId=query_execution_id
        )
        state = response["QueryExecution"]["Status"]["State"]
        
        if state in ["SUCCEEDED", "FAILED", "CANCELLED"]:
            return state
        
        time.sleep(1)

- return query results

In [24]:
def get_query_results(query_execution_id: str) -> pd.DataFrame:
    """
    Retrieve results from a completed Athena query as a DataFrame.
    """
    # Get results
    response = athena_client.get_query_results(
        QueryExecutionId=query_execution_id
    )
    
    # Parse results into DataFrame
    rows = response["ResultSet"]["Rows"]
    
    # First row is headers
    headers = [col["VarCharValue"] for col in rows[0]["Data"]]
    
    # Remaining rows are data
    data = []
    for row in rows[1:]:
        data.append([col.get("VarCharValue", None) for col in row["Data"]])
    
    return pd.DataFrame(data, columns=headers)

In [25]:
def query_athena(query: str, database: str = GLUE_DATABASE_NAME) -> pd.DataFrame:
    """
    Convenience function: Run query and return results as DataFrame.
    """
    print(f"Executing query...")
    
    # Start query
    query_id = run_athena_query(query, database, ATHENA_OUTPUT_LOCATION)
    
    # Wait for completion
    state = wait_for_query(query_id)
    
    if state == "SUCCEEDED":
        print(f"Query completed")
        return get_query_results(query_id)
    else:
        print(f"✗ Query {state}")
        # Get error message
        response = athena_client.get_query_execution(QueryExecutionId=query_id)
        error = response["QueryExecution"]["Status"].get("StateChangeReason", "Unknown error")
        print(f" Error: {error}")
        return pd.DataFrame()

- count total rows

In [26]:
# Use the table name discovered earlier
TABLE_NAME = "raw_parquet"

query = f"SELECT COUNT(*) as total_rows FROM {TABLE_NAME}"
print(f"Query: {query}\n")

result = query_athena(query)
result

Query: SELECT COUNT(*) as total_rows FROM raw_parquet

Executing query...
Query completed


,total_rows
0,12000


In [27]:
# first 10 rows
query = f"SELECT * FROM {TABLE_NAME} LIMIT 10"
print(f"Query: {query}\n")

result = query_athena(query)
result

Query: SELECT * FROM raw_parquet LIMIT 10

Executing query...
Query completed


,state_fips,date,unrate,payems,civpart,emratio,u6rate,awhman,ahetpi,cpiaucsl,...,walcl,spread_10y_2y,spread_10y_3m,cpi_yoy,state_name,b19013_001e,b01003_001e,b25077_001e,b25064_001e,b17001_002e
0,1,1104537600000000000,5.3,132781,65.8,62.4,9.2,40.7,15.9,191.6,...,807262.0,0.8499999999999996,1.63,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
1,1,1107216000000000000,5.4,133033,65.9,62.4,9.2,40.7,15.93,192.4,...,804576.0,0.7700000000000005,1.6000000000000003,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
2,1,1109635200000000000,5.2,133152,65.9,62.4,9.1,40.4,15.97,193.1,...,807551.0,0.7000000000000002,1.71,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
3,1,1112313600000000000,5.2,133519,66.1,62.7,9.0,40.4,16.01,193.7,...,809797.0,0.5499999999999998,1.31,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
4,1,1114905600000000000,5.1,133689,66.1,62.8,8.9,40.4,16.03,193.6,...,810160.0,0.3999999999999999,1.0099999999999998,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
5,1,1117584000000000000,5.0,133947,66.1,62.7,9.0,40.4,16.08,193.7,...,810082.0,0.2799999999999998,0.81,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
6,1,1120176000000000000,5.0,134300,66.1,62.8,8.8,40.5,16.15,194.9,...,814259.0,0.2600000000000007,0.8600000000000003,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
7,1,1122854400000000000,4.9,134503,66.2,62.9,8.9,40.5,16.17,196.1,...,823237.0,0.1799999999999997,0.4999999999999995,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
8,1,1125532800000000000,5.0,134554,66.1,62.8,9.0,40.7,16.19,198.8,...,823771.0,0.1600000000000001,0.79,None,None,42081.0,4712651.0,117600.0,644.0,786544.0
9,1,1128124800000000000,5.0,134643,66.1,62.8,8.7,41.0,16.28,199.1,...,819105.0,0.1699999999999999,0.5900000000000003,None,None,42081.0,4712651.0,117600.0,644.0,786544.0


In [29]:
# date range and unique dates check
query = f"""
SELECT 
    MIN(date) as earliest_date,
    MAX(date) as latest_date,
    COUNT(DISTINCT date) as unique_dates
FROM {TABLE_NAME}
"""
print(f"Query: {query}")

result = query_athena(query)
result

Query: 
SELECT 
    MIN(date) as earliest_date,
    MAX(date) as latest_date,
    COUNT(DISTINCT date) as unique_dates
FROM raw_parquet

Executing query...
Query completed


,earliest_date,latest_date,unique_dates
0,1104537600000000000,1733011200000000000,240


In [30]:
# rows per state_fips (first 10 states)
query = f"""
SELECT 
    state_fips,
    COUNT(*) as row_count
FROM {TABLE_NAME}
GROUP BY state_fips
ORDER BY state_fips
LIMIT 10
"""
print(f"Query: {query}")

result = query_athena(query)
result

Query: 
SELECT 
    state_fips,
    COUNT(*) as row_count
FROM raw_parquet
GROUP BY state_fips
ORDER BY state_fips
LIMIT 10

Executing query...
Query completed


,state_fips,row_count
0,1,240
1,2,240
2,4,240
3,5,240
4,6,240
5,8,240
6,9,240
7,10,240
8,12,240
9,13,240


## clean up

In [ ]:
# # 1. Delete the crawler
# try:
#     glue_client.delete_crawler(Name=GLUE_CRAWLER_NAME)
#     print(f"✓ Deleted crawler: {GLUE_CRAWLER_NAME}")
# except ClientError as e:
#     print(f"  Crawler delete: {e}")

# # 2. Delete the tables
# for table_name in ["state_month_full_parquet", "state_month_full_csv"]:
#     try:
#         glue_client.delete_table(
#             DatabaseName=GLUE_DATABASE_NAME,
#             Name=table_name
#         )
#         print(f"✓ Deleted table: {table_name}")
#     except ClientError as e:
#         print(f"  Table delete: {e}")

# # 3. Delete the database
# try:
#     glue_client.delete_database(Name=GLUE_DATABASE_NAME)
#     print(f"✓ Deleted database: {GLUE_DATABASE_NAME}")
# except ClientError as e:
#     print(f"  Database delete: {e}")

# print("\n✓ Cleanup complete — ready to restart notebook")

✓ Deleted crawler: nfci-raw-data-crawler
✓ Deleted table: state_month_full_parquet
  Table delete: An error occurred (EntityNotFoundException) when calling the DeleteTable operation: Table state_month_full_csv not found.
✓ Deleted database: nfci_database

✓ Cleanup complete — ready to restart notebook
